In [1]:
%matplotlib inline

# Необходимые библиотеки:

import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import IPython
import nltk
import re

# функция train_test_split для разбиения базы на тренировочную и тестовую
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/Anastasia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

В данном примере стоит задач по наименованию товара присвоить ему определенную группу - задача множественной классификации.
Загружаем данные с уже проставленными группами для обучения модели:

In [2]:
df = pd.read_excel('Учебная база.xlsx')
df.sample(10)

,группа отчета,Категория,category_1,category_2,category_3,category_4,category_5,product_id,retailer_id,title,mpn,status,brand,store,rating,sales,gmv,stock_atnow,url,image
242064,Приборы для бритья,Бытовая техника; Техника для красоты и здоровь...,Бытовая техника,Техника для красоты и здоровья,Электробритвы и аксессуары,Электробритвы,NaN,32119565,240453542,Электробритва Xiaomi Mijia Electric Shaver S30...,NaN,deleted,Xiaomi,65198,NaN,NaN,NaN,3,https://www.ozon.ru/context/detail/id/240453542/,https://cdn1.ozone.ru/s3/multimedia-2/60163577...
45614,Увлажнители воздуха,Бытовая техника; Климатическая техника; Увлажн...,Бытовая техника,Климатическая техника,Увлажнители воздуха и аромадиффузоры,NaN,NaN,4915759,173732306,"Увлажнитель воздуха Electrolux EHU-3710D, черный",НС-1073571,available,Electrolux,42957,5.0,NaN,NaN,300,https://www.ozon.ru/context/detail/id/173732306/,https://cdn1.ozone.ru/s3/multimedia-x/60125629...
350040,Йогуртницы,Бытовая техника; Техника для кухни; Техника дл...,Бытовая техника,Техника для кухни,Техника для приготовления десертов,Йогуртницы,NaN,51070790,284051905,Йогуртница Redmond RYM-M5406,5055323625572,deleted,Redmond,172559,NaN,NaN,NaN,1,https://www.ozon.ru/context/detail/id/284051905/,https://cdn1.ozone.ru/s3/multimedia-w/60733038...
28931,Кухонные принадлежности,Бытовая техника; Техника для кухни; Прочая кух...,Бытовая техника,Техника для кухни,Прочая кухонная техника,Кулинарные термометры,NaN,18254683,205691398,Термометр кулинарный для мяса,NaN,deleted,Vibrosa,94412,NaN,NaN,NaN,3037,https://www.ozon.ru/context/detail/id/205691398/,https://cdn1.ozone.ru/s3/multimedia-k/60129479...
419146,Накопительные водонагреватели,Бытовая техника; Климатическая техника; Водона...,Бытовая техника,Климатическая техника,Водонагреватели,Накопительные,NaN,80410056,348997111,Водонагреватель накопительный электрический Th...,725873,deleted,Thermex,239864,NaN,NaN,NaN,1,https://www.ozon.ru/context/detail/id/348997111/,https://cdn1.ozone.ru/s3/multimedia-j/61211030...
147586,Аксессуары,Бытовая техника; Техника для дома; Пылесосы и ...,Бытовая техника,Техника для дома,Пылесосы и аксессуары,Аксессуары для пылесосов,Пылесборники,38852426,258953786,Мешки-пылесборники ZOOM синтетические 5 шт для...,NaN,deleted,Zoom,34266,5.0,NaN,NaN,10,https://www.ozon.ru/context/detail/id/258953786/,https://cdn1.ozone.ru/s3/multimedia-g/60608194...
406404,Стационарные блендеры,"Бытовая техника; Техника для кухни; Миксеры, б...",Бытовая техника,Техника для кухни,"Миксеры, блендеры и измельчители",Блендеры,Стационарные,77856204,348539023,"Стационарный блендер Gorenje B800ORAB, черный",NaN,deleted,Gorenje,239874,NaN,NaN,NaN,1,https://www.ozon.ru/context/detail/id/348539023/,https://cdn1.ozone.ru/s3/multimedia-n/61167691...
49824,Стайлеры,Бытовая техника; Техника для красоты и здоровь...,Бытовая техника,Техника для красоты и здоровья,Выпрямители для волос,NaN,NaN,49800307,283043111,"Выпрямитель для волос IRIT IR-3150 ,розовый",NaN,deleted,IRIT,65590,NaN,NaN,NaN,193,https://www.ozon.ru/context/detail/id/283043111/,https://cdn1.ozone.ru/s3/multimedia-7/60726948...
162695,Хлебопечки,Бытовая техника; Техника для кухни; Печи и гри...,Бытовая техника,Техника для кухни,Печи и грили,Хлебопечки,NaN,56568542,289432296,"Хлебопечка Centek CT-1406, белый, черный",NaN,available,Centek,170659,NaN,NaN,NaN,10,https://www.ozon.ru/context/detail/id/289432296/,https://cdn1.ozone.ru/s3/multimedia-0/60277685...
54590,ДУХОВКИ,Бытовая техника; Крупная бытовая техника; Духо...,Бытовая техника,Крупная бытовая техника,Духовые шкафы,Электрические,NaN,3717,150886470,Электрический Духовой шкаф Simfer B6EO18017,B6EO18017,available,Simfer,8404,5.0,NaN,NaN,102,https://www.ozon.ru/context/detail/id/150886470/,https://cdn1.ozone.ru/s3/multimedia-1021701359...


Создаем объект типа Series для нумерации групп товаров:

In [3]:
s_category = pd.Series(data = [x for x in range(len(df['группа отчета'].unique()))], 
                    index = df['группа отчета'].unique().tolist(), name='targets')
s_category

Роботы                        0
Стайлеры                      1
Кофемашины                    2
ВАРОЧНЫЕ ПОВЕРХНОСТИ          3
Грили электрические           4
                             ..
Ручной отпариватель          85
Вертикальный отпариватель    86
Водонагреватели прочие       87
кондиционеры                 88
Сушилки для обуви            89
Name: targets, Length: 90, dtype: int64

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455490 entries, 0 to 455489
Data columns (total 20 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   группа отчета  455490 non-null  object 
 1   Категория      455490 non-null  object 
 2   category_1     455490 non-null  object 
 3   category_2     455490 non-null  object 
 4   category_3     455485 non-null  object 
 5   category_4     413678 non-null  object 
 6   category_5     78659 non-null   object 
 7   product_id     455490 non-null  int64  
 8   retailer_id    455490 non-null  int64  
 9   title          455490 non-null  object 
 10  mpn            149198 non-null  object 
 11  status         455398 non-null  object 
 12  brand          455432 non-null  object 
 13  store          455490 non-null  object 
 14  rating         74340 non-null   float64
 15  sales          22708 non-null   float64
 16  gmv            22708 non-null   float64
 17  stock_atnow    455490 non-nul

In [5]:
df = df.set_index('группа отчета').join(s_category)
display(df.info(), df.sample(10))

<class 'pandas.core.frame.DataFrame'>
Index: 455490 entries, Аксессуары to швейные машинки
Data columns (total 20 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Категория    455490 non-null  object 
 1   category_1   455490 non-null  object 
 2   category_2   455490 non-null  object 
 3   category_3   455485 non-null  object 
 4   category_4   413678 non-null  object 
 5   category_5   78659 non-null   object 
 6   product_id   455490 non-null  int64  
 7   retailer_id  455490 non-null  int64  
 8   title        455490 non-null  object 
 9   mpn          149198 non-null  object 
 10  status       455398 non-null  object 
 11  brand        455432 non-null  object 
 12  store        455490 non-null  object 
 13  rating       74340 non-null   float64
 14  sales        22708 non-null   float64
 15  gmv          22708 non-null   float64
 16  stock_atnow  455490 non-null  int64  
 17  url          455490 non-null  object 
 18  image      

None

,Категория,category_1,category_2,category_3,category_4,category_5,product_id,retailer_id,title,mpn,status,brand,store,rating,sales,gmv,stock_atnow,url,image,targets
Весы напольные,Бытовая техника; Техника для красоты и здоровь...,Бытовая техника,Техника для красоты и здоровья,Напольные весы,Электронные,NaN,6539194,178593580,Напольные весы Mayer & Boch MB-20908,NaN,deleted,Mayer & Boch,52966,NaN,NaN,NaN,10,https://www.ozon.ru/context/detail/id/178593580/,https://cdn1.ozone.ru/s3/multimedia-1022790506...,13
Увлажнители воздуха,Бытовая техника; Климатическая техника; Увлажн...,Бытовая техника,Климатическая техника,Увлажнители воздуха и аромадиффузоры,NaN,NaN,200480,155371735,Увлажнитель воздуха Scarlett Увлажнитель возду...,SC-AH986E07,available,Scarlett,25000,4.713636,613.0,1933389.2,460,https://www.ozon.ru/context/detail/id/155371735/,https://cdn1.ozone.ru/s3/multimedia-n/60361594...,22
Разное,Бытовая техника; Климатическая техника; Погодн...,Бытовая техника,Климатическая техника,Погодные станции и датчики,Датчики погодной станции,NaN,891868,169356706,Датчик погодной станции Oregon Scientific,UVN800,available,Oregon Scientific,3854,NaN,NaN,NaN,5,https://www.ozon.ru/context/detail/id/169356706/,https://cdn1.ozone.ru/s3/multimedia-1021541465...,12
Разное,Бытовая техника; Техника для красоты и здоровь...,Бытовая техника,Техника для красоты и здоровья,Электрические зубные щетки и аксессуары,Зубные щетки,NaN,30544346,234642031,Электрическая зубная щетка SONIC TOOTHBRUSH X-...,NaN,available,SONIC TOOTHBRUSH,125529,NaN,NaN,NaN,999,https://www.ozon.ru/context/detail/id/234642031/,https://cdn1.ozone.ru/s3/multimedia-j/60256220...,12
Плитки электрические,Бытовая техника; Техника для кухни; Настольные...,Бытовая техника,Техника для кухни,Настольные плиты,Настольные электрические плиты,NaN,64226020,306317264,"Электрическая Настольная плита Atlanta, белый",77.858@29897,deleted,Atlanta,98341,NaN,NaN,NaN,1,https://www.ozon.ru/context/detail/id/306317264/,https://cdn1.ozone.ru/s3/multimedia-8/60603741...,68
Электрокамины,Бытовая техника; Климатическая техника; Обогре...,Бытовая техника,Климатическая техника,Обогреватели и тепловентиляторы,Электрокамины,NaN,66261088,308313616,Электрокамин Siena c очагом Beverly 1000,NaN,available,Real Flame,164344,NaN,NaN,NaN,2,https://www.ozon.ru/context/detail/id/308313616/,https://cdn1.ozone.ru/s3/multimedia-r/60900032...,47
Разное,Бытовая техника; Техника для дома; Швейные маш...,Бытовая техника,Техника для дома,Швейные машины и аксессуары,Оверлоки и распошивальные машины,NaN,63290767,306442097,Оверлок Juki,MO 654 DEN,deleted,Juki,ozon,NaN,NaN,NaN,1,https://www.ozon.ru/context/detail/id/306442097/,https://cdn1.ozone.ru/s3/multimedia-1021370177...,12
Увлажнители воздуха,Бытовая техника; Климатическая техника; Увлажн...,Бытовая техника,Климатическая техника,Увлажнители воздуха и аромадиффузоры,NaN,NaN,34212486,243471219,Увлажнитель воздуха TEENRA NM-Grey and 8 filte...,NaN,available,TEENRA,138236,NaN,NaN,NaN,9729,https://www.ozon.ru/context/detail/id/243471219/,https://cdn1.ozone.ru/s3/multimedia-y/60440876...,22
Электрокамины,Бытовая техника; Климатическая техника; Обогре...,Бытовая техника,Климатическая техника,Обогреватели и тепловентиляторы,Электрокамины,NaN,66261170,313883535,3D электрокамин Dacota WT с Olympic,NaN,available,Real Flame,164344,NaN,NaN,NaN,2,https://www.ozon.ru/context/detail/id/313883535/,https://cdn1.ozone.ru/s3/multimedia-r/60928636...,47
СВЧ,Бытовая техника; Техника для кухни; Печи и гри...,Бытовая техника,Техника для кухни,Печи и грили,Микроволновые печи,NaN,68704741,320949904,Микроволновая печь BBK 20MWG-740M/S,NaN,available,BBK,150005,NaN,NaN,NaN,20,https://www.ozon.ru/context/detail/id/320949904/,https://cdn1.ozone.ru/s3/multimedia-4/61023270...,34


In [6]:
df = df.reset_index()
df.sample(5)

,index,Категория,category_1,category_2,category_3,category_4,category_5,product_id,retailer_id,title,...,status,brand,store,rating,sales,gmv,stock_atnow,url,image,targets
6365,Аксессуары,Бытовая техника; Техника для кухни; Кофеварки ...,Бытовая техника,Техника для кухни,Кофеварки и кофемашины,Аксессуары для кофемашин и кофеварок,Запчасти и аксессуары,30468257,234382641,Многоразовые кофейные капсулы разноцветные 2ш...,...,available,TEENRA,126359,NaN,NaN,NaN,7085,https://www.ozon.ru/context/detail/id/234382641/,https://cdn1.ozone.ru/s3/multimedia-f/60373949...,31
434287,Чайники,Бытовая техника; Техника для кухни; Электричес...,Бытовая техника,Техника для кухни,Электрические чайники и термопоты,Чайники,NaN,68951272,321108910,Электрический чайник Kitfort Электрический чай...,...,deleted,Kitfort,ozon,NaN,NaN,NaN,1,https://www.ozon.ru/context/detail/id/321108910/,https://cdn1.ozone.ru/s3/multimedia-b/60098987...,24
33320,Аксессуары,Бытовая техника; Техника для красоты и здоровь...,Бытовая техника,Техника для красоты и здоровья,Машинки для стрижки волос и аксессуары,Насадки,NaN,23325060,218054041,92М Бритвенная кассета для бритвы Braun 9 сери...,...,deleted,Braun,27402,5.000000,NaN,NaN,6,https://www.ozon.ru/context/detail/id/218054041/,https://cdn1.ozone.ru/s3/multimedia-f/60335974...,31
271146,Прочие обогреватели,Бытовая техника; Климатическая техника; Обогре...,Бытовая техника,Климатическая техника,Обогреватели и тепловентиляторы,Обогреватели,NaN,65596139,311426505,Кварцевый настенный обогреватель Equation 1200 Вт,...,available,equation,166291,NaN,NaN,NaN,300,https://www.ozon.ru/context/detail/id/311426505/,https://cdn1.ozone.ru/s3/multimedia-b/60925184...,15
253456,Приборы по уходу за кожей и ногтями,Бытовая техника; Техника для красоты и здоровь...,Бытовая техника,Техника для красоты и здоровья,Массажное оборудование и аксессуары,Массажные подушки,NaN,24896680,220612684,Центр распродаж Электрический массажер для н...,...,available,Центр распродаж,38117,4.566667,NaN,NaN,660,https://www.ozon.ru/context/detail/id/220612684/,https://cdn1.ozone.ru/s3/multimedia-8/60282978...,10


Создаем учебную и тестовую базы:

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    df['title'], df['targets'], random_state=0)

In [8]:
print("форма массива X_train: {}".format(X_train.shape)) 
print("форма массива y_train: {}".format(y_train.shape))

форма массива X_train: (341617,)
форма массива y_train: (341617,)


In [9]:
print("форма массива X_test: {}".format(X_test.shape)) 
print("форма массива y_test: {}".format(y_test.shape))

форма массива X_test: (113873,)
форма массива y_test: (113873,)


In [10]:
X_train

123846    Электрический Духовой шкаф Electrolux OKC5H50W...
278464    Ручной пылесос Xiaomi Shunzao Z1 (белый), русс...
107797     Измельчитель бытовых отходов Bone Crusher BC 810
357409    Стационарный блендер Electrolux ESB 2700, сере...
83157     Электрическая варочная панель Hansa BHC66500, ...
                                ...                        
152315    Кондиционер Tosot T09H-SnN/I/T09H-SnN/O серии ...
439107        Паровая швабра Scarlett SC-SM31B01, бирюзовый
117952     Вытяжка Elikor Аметист S4 90Н-700-Э4Д нерж/тонир
435829                  Электрический чайник Kitfort KT-670
305711    Сушилка для рук электрическая BXG-JET-3100А с ...
Name: title, Length: 341617, dtype: object

Функция, обрабатывающая наши наименования. Здесь мы убираем все слова из иных букв, кроме киррилицы, токинезируем и записываем заново в нижнем регистре.

In [11]:
def preprocess_text(texts):
    regex = re.compile('[^а-я А-Я]')
    sent_detector = nltk.data.load('tokenizers/punkt/PY3/russian.pickle')
    word_tokens = word_tokenize(texts, language='russian')
    preprocess_text = []
    for i in word_tokens:
        text = i.lower()
        text = regex.sub('', text)
        preprocess_text.append(text)
    preprocess_text = ' '.join([w for w in preprocess_text if w != ''])
    return preprocess_text

Тестируем:

In [12]:
abc = 'Русский язык Russian 126 писать и есть салат one egg'
preprocess_text(abc)

'русский язык писать и есть салат'

Теперь применяем функцию к тренировочной и тестовой базам:

In [13]:
X_train_preproc = X_train.apply(preprocess_text)
X_train_preproc

123846    электрический духовой шкаф утапливаемые перекл...
278464                  ручной пылесос белый русская версия
107797                         измельчитель бытовых отходов
357409                     стационарный блендер серебристый
83157                  электрическая варочная панель черный
                                ...                        
152315                                    кондиционер серии
439107                             паровая швабра бирюзовый
117952                        вытяжка аметист нэд нержтонир
435829                                 электрический чайник
305711    сушилка для рук электрическая а с функцией уль...
Name: title, Length: 341617, dtype: object

In [14]:
X_test_preproc = X_test.apply(preprocess_text)
X_test_preproc

176582                                 кулинарный термометр
57309                                 стандартный фильтр шт
275808             вертикальный пылесос кт черный сиреневый
167617    электронные кухонные весы весы кухонные кг эле...
183361        машинка для стрижки набор для стрижки золотой
                                ...                        
103762                     напольные весы камни белый серый
402655                         фен для волос красный черный
356050                      стационарный блендер кт голубой
339364                        соковыжималка уцененный товар
99075                                 напольные весы черный
Name: title, Length: 113873, dtype: object

Далее необходимо текстовую информацию перевести в числовую. Для этого будем применять метод "мешок слов", которых соберет совокупность всех слов в выборке и создаст матрицу с количеством колонок, равным числу слов, в строках будет отмечено число вхождения слова в соответвтующее наименование:

In [15]:
vectorizer_tfidf = TfidfVectorizer()

In [16]:
def bow(vectorizer, trains, tests):
    train_bow = vectorizer.fit_transform(trains)
    test_bow = vectorizer.transform(tests)
    
    return train_bow, test_bow

In [17]:
X_train_preproc_bow, X_test_preproc_bow = bow(vectorizer_tfidf, X_train_preproc, X_test_preproc)

У нас получилась таблица такого размера:

In [18]:
X_train_preproc_bow.shape

(341617, 16949)

Обучим нашу модель с применением LinearSVC - метод опорных вектором для классификации:

In [19]:
clf_svc = LinearSVC()
clf_svc.fit(X_train_preproc_bow, y_train)

LinearSVC()

В результате модель выдает точность почти 97,7% на тестовой выборке:

In [20]:
accuracy_score(clf_svc.predict(X_test_preproc_bow), y_test)

0.9768865314868318

Сохраняем предсказанный результат в переменную:

In [21]:
y_test_predict = clf_svc.predict(X_test_preproc_bow)
y_test_predict

array([51, 31,  6, ..., 56, 52, 13])

Далее идут манипуляции для присоединения групп к тестовой выборке, чтобы визуально посмотреть, все ли сработало:

In [22]:
test_df = pd.DataFrame(X_test)
test_df['targets'] = y_test_predict
test_df

,title,targets
176582,Кулинарный термометр meltbox,51
57309,"Стандартный фильтр EURO Clean EUR HS-16-M0011,...",31
275808,"Вертикальный пылесос Kitfort КТ-584, черный, с...",6
167617,Электронные Кухонные весы Sakura Весы кухонные...,64
183361,"Машинка для стрижки, Набор для стрижки Goods H...",42
...,...,...
103762,"Напольные весы REDMOND RS-752 Камни, белый, серый",13
402655,"Фен для волос Panasonic EH-NA65-K865, красный,...",14
356050,"Стационарный блендер Kitfort КТ-1341-2, голубой",56
339364,"Соковыжималка Oursson JM6001/DC, Dark Cherry У...",52


In [23]:
s_category = pd.DataFrame(s_category).reset_index()
s_category

,index,targets
0,Роботы,0
1,Стайлеры,1
2,Кофемашины,2
3,ВАРОЧНЫЕ ПОВЕРХНОСТИ,3
4,Грили электрические,4
...,...,...
85,Ручной отпариватель,85
86,Вертикальный отпариватель,86
87,Водонагреватели прочие,87
88,кондиционеры,88


In [24]:
test_df = test_df.merge(s_category, on = 'targets', how = 'left')
test_df

,title,targets,index
0,Кулинарный термометр meltbox,51,Кухонные принадлежности
1,"Стандартный фильтр EURO Clean EUR HS-16-M0011,...",31,Аксессуары
2,"Вертикальный пылесос Kitfort КТ-584, черный, с...",6,Пылесосы
3,Электронные Кухонные весы Sakura Весы кухонные...,64,Кухонные весы
4,"Машинка для стрижки, Набор для стрижки Goods H...",42,Машинки для стрижки
...,...,...,...
113868,"Напольные весы REDMOND RS-752 Камни, белый, серый",13,Весы напольные
113869,"Фен для волос Panasonic EH-NA65-K865, красный,...",14,Фены
113870,"Стационарный блендер Kitfort КТ-1341-2, голубой",56,Стационарные блендеры
113871,"Соковыжималка Oursson JM6001/DC, Dark Cherry У...",52,Соковыжималки


In [25]:
test_df.sample(20)

,title,targets,index
56138,"Балкон (полка,Ящик) двери холодильника Минск А...",31,Аксессуары
9915,Take Easy Мини кондиционер Arctic Air,76,Кондиционеры
44650,Щипцы-плойка для завивки волос BaByliss Pro To...,1,Стайлеры
11040,Робот-пылесос Homberg HB13923 . Уцененный то...,0,Роботы
63171,Ирригатор для полости рта портативный для чист...,12,Разное
113366,Подшипник 6204 для стиральной машины (20x47x14),31,Аксессуары
2273,Кулер для воды AEL 390 с компрессорным охлажде...,12,Разное
90398,Швейная машина Singer CONFIDENCE 7640Q,25,швейные машинки
62507,Многоразовые капсулы для кофемашины Dolce Gust...,31,Аксессуары
13355,Весы ювелирные кухонные KAMEEL KLA-2 (2000/0.1...,64,Кухонные весы
